<a href="https://colab.research.google.com/github/arpan-sharma/Machine-learning_projects/blob/main/using_tensorflow_chatbot_funny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install tflearn

In [18]:
#Import Libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer
steammer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#Loading our JSON Data

import json
with open('/content/drive/MyDrive/chat_bot/intents_fun.json') as file:
    data = json.load(file)

In [11]:
#Extracting Data
words = []
labels = []
docs_x = []
docs_y = []

In [16]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [19]:
#Word Stemming

words = [steammer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


In [21]:
#Bag of Words
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [steammer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [22]:
#we will convert our training data and output to numpy arrays.
training = numpy.array(training)
output = numpy.array(output)


In [ ]:
#Developing a Model
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


In [25]:
#Training & Saving the Model
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 10999  | total loss: 0.00075 | time: 0.037s
| Adam | epoch: 1000 | loss: 0.00075 - acc: 1.0000 -- iter: 80/86
Training Step: 11000  | total loss: 0.00079 | time: 0.040s
| Adam | epoch: 1000 | loss: 0.00079 - acc: 1.0000 -- iter: 86/86
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [28]:
#Making Predictions
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [steammer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: what is your name
I'm Lalu also known as:Darubazz.
You: one bollywood movie dailog
Bade se bada business … paise se nahi, ek bade idea se bada hota hai
You: again dailog
Kuchh Bhi Karne Ka But, Meri Ego Ko Hurt Nahi Karne Ka.
You: fir se ek dailog
Hum hi hum hai toh kya hum hai … tum hi tum ho toh kya tum ho
You: who is you
Hello!
You: chite ke chal dailog 
Himmat batai nahi … dikhai jaati hai
You: one word for you
Talk to you later
You: if you are human what you say
Hello!
You: why you say hello
Hello!
You: movie dailog
Jis mohabbat mein deewangi na ho … woh mohabbat hi nahi
You: dailog
Dosti Ka Ek Usool He, Madam: No Sorry, No Thank You…
You: gali bak
Sad to see you go :(
You: galiyan
Sad to see you go :(
You: absuse
Sad to see you go :(
You: madarchod
Goodbye!
You: gandu
Sad to see you go :(
You: quit
